In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [92]:
email_opened = pd.read_csv("email_opened_table.csv")
email_table = pd.read_csv("email_table.csv")
link_clicked = pd.read_csv("link_clicked_table.csv")

In [93]:
email_table.isnull().sum()

email_id               0
email_text             0
email_version          0
hour                   0
weekday                0
user_country           0
user_past_purchases    0
dtype: int64

In [94]:
email_opened.isnull().sum()

email_id    0
dtype: int64

In [95]:
link_clicked.isnull().sum()

email_id    0
dtype: int64

In [96]:
email_opened.head()

,email_id
0,284534
1,609056
2,220820
3,905936
4,164034


In [97]:
email_table.head()

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases
0,85120,short_email,personalized,2,Sunday,US,5
1,966622,long_email,personalized,12,Sunday,UK,2
2,777221,long_email,personalized,11,Wednesday,US,2
3,493711,short_email,generic,6,Monday,UK,1
4,106887,long_email,generic,14,Monday,US,6


In [98]:
link_clicked.head()

,email_id
0,609056
1,870980
2,935124
3,158501
4,177561


In [99]:
email_table['opened'] = email_table['email_id'].isin(email_opened['email_id']).astype(int)
email_table['clicked'] = email_table['email_id'].isin(link_clicked['email_id']).astype(int)

In [100]:
email_table.head(5)

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,opened,clicked
0,85120,short_email,personalized,2,Sunday,US,5,0,0
1,966622,long_email,personalized,12,Sunday,UK,2,1,1
2,777221,long_email,personalized,11,Wednesday,US,2,0,0
3,493711,short_email,generic,6,Monday,UK,1,0,0
4,106887,long_email,generic,14,Monday,US,6,0,0


In [101]:
email_table.tail(5)

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,opened,clicked
99995,803504,short_email,personalized,4,Monday,US,5,0,0
99996,899722,long_email,personalized,7,Saturday,US,1,0,0
99997,449610,long_email,personalized,11,Saturday,UK,6,0,0
99998,72497,short_email,generic,10,Monday,UK,0,0,0
99999,348333,long_email,personalized,8,Sunday,UK,1,1,1


In [102]:
## Basic Statistics

open_rate = email_table['opened'].mean() * 100
ctr = email_table['clicked'].mean() * 100
print(f"Open Rate: {open_rate:.2f}%")
print(f"Click-Through Rate: {ctr:.2f}%")

Open Rate: 10.35%
Click-Through Rate: 2.12%


In [11]:
def plot_bar(feature, target):
    plt.figure(figsize=(10, 6))
    sns.barplot(x=feature, y=target, data=email_table)
    plt.title(f'{target.capitalize()} Rate by {feature.capitalize()}')
    plt.xticks(rotation=45)
    plt.savefig(f'{target}_by_{feature}.png')
    plt.close()

for feature in ['email_text', 'email_version', 'user_country', 'weekday']:
    plot_bar(feature, 'opened')
    plot_bar(feature, 'clicked')

In [12]:
email_table['hour'] = email_table['hour'].astype(int)
plot_bar('hour', 'opened')
plot_bar('hour', 'clicked')

In [103]:
label_encoder_gender = LabelEncoder()
email_table['email_text'] = label_encoder_gender.fit_transform(email_table['email_text'])

In [104]:
email_table

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,opened,clicked
0,85120,1,personalized,2,Sunday,US,5,0,0
1,966622,0,personalized,12,Sunday,UK,2,1,1
2,777221,0,personalized,11,Wednesday,US,2,0,0
3,493711,1,generic,6,Monday,UK,1,0,0
4,106887,0,generic,14,Monday,US,6,0,0
...,...,...,...,...,...,...,...,...,...
99995,803504,1,personalized,4,Monday,US,5,0,0
99996,899722,0,personalized,7,Saturday,US,1,0,0
99997,449610,0,personalized,11,Saturday,UK,6,0,0
99998,72497,1,generic,10,Monday,UK,0,0,0


In [105]:
onehot_encoder_country = OneHotEncoder()
country_encoder = onehot_encoder_country.fit_transform(email_table[['user_country']])
country_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100000 stored elements and shape (100000, 4)>

In [106]:
onehot_encoder_country.get_feature_names_out(['user_country'])

array(['user_country_ES', 'user_country_FR', 'user_country_UK',
       'user_country_US'], dtype=object)

In [107]:
country_encoded_df = pd.DataFrame(country_encoder.toarray(),columns=onehot_encoder_country.get_feature_names_out(['user_country'])).astype(int)
country_encoded_df.head()

,user_country_ES,user_country_FR,user_country_UK,user_country_US
0,0,0,0,1
1,0,0,1,0
2,0,0,0,1
3,0,0,1,0
4,0,0,0,1


In [108]:
email_table = pd.concat([email_table.drop(['user_country'],axis=1),country_encoded_df],axis=1)

In [109]:
email_table

,email_id,email_text,email_version,hour,weekday,user_past_purchases,opened,clicked,user_country_ES,user_country_FR,user_country_UK,user_country_US
0,85120,1,personalized,2,Sunday,5,0,0,0,0,0,1
1,966622,0,personalized,12,Sunday,2,1,1,0,0,1,0
2,777221,0,personalized,11,Wednesday,2,0,0,0,0,0,1
3,493711,1,generic,6,Monday,1,0,0,0,0,1,0
4,106887,0,generic,14,Monday,6,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,803504,1,personalized,4,Monday,5,0,0,0,0,0,1
99996,899722,0,personalized,7,Saturday,1,0,0,0,0,0,1
99997,449610,0,personalized,11,Saturday,6,0,0,0,0,1,0
99998,72497,1,generic,10,Monday,0,0,0,0,0,1,0


In [110]:
onehot_encoder_week = OneHotEncoder()
week_encoder = onehot_encoder_week.fit_transform(email_table[['weekday']])
week_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100000 stored elements and shape (100000, 7)>

In [111]:
onehot_encoder_week.get_feature_names_out(['weekday'])

array(['weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday'], dtype=object)

In [112]:
week_encoded_df = pd.DataFrame(week_encoder.toarray(),columns=onehot_encoder_week.get_feature_names_out(['weekday'])).astype(int)
week_encoded_df.head()

,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0


In [113]:
email_table = pd.concat([email_table.drop(['weekday'],axis=1),week_encoded_df],axis=1)

In [114]:
email_table

,email_id,email_text,email_version,hour,user_past_purchases,opened,clicked,user_country_ES,user_country_FR,user_country_UK,user_country_US,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,85120,1,personalized,2,5,0,0,0,0,0,1,0,0,0,1,0,0,0
1,966622,0,personalized,12,2,1,1,0,0,1,0,0,0,0,1,0,0,0
2,777221,0,personalized,11,2,0,0,0,0,0,1,0,0,0,0,0,0,1
3,493711,1,generic,6,1,0,0,0,0,1,0,0,1,0,0,0,0,0
4,106887,0,generic,14,6,0,0,0,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,803504,1,personalized,4,5,0,0,0,0,0,1,0,1,0,0,0,0,0
99996,899722,0,personalized,7,1,0,0,0,0,0,1,0,0,1,0,0,0,0
99997,449610,0,personalized,11,6,0,0,0,0,1,0,0,0,1,0,0,0,0
99998,72497,1,generic,10,0,0,0,0,0,1,0,0,1,0,0,0,0,0


In [115]:
label_encoder_version = LabelEncoder()
email_table['email_version'] = label_encoder_version.fit_transform(email_table['email_version'])
email_table.head()

,email_id,email_text,email_version,hour,user_past_purchases,opened,clicked,user_country_ES,user_country_FR,user_country_UK,user_country_US,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,85120,1,1,2,5,0,0,0,0,0,1,0,0,0,1,0,0,0
1,966622,0,1,12,2,1,1,0,0,1,0,0,0,0,1,0,0,0
2,777221,0,1,11,2,0,0,0,0,0,1,0,0,0,0,0,0,1
3,493711,1,0,6,1,0,0,0,0,1,0,0,1,0,0,0,0,0
4,106887,0,0,14,6,0,0,0,0,0,1,0,1,0,0,0,0,0


In [116]:
# corr_columns = [
#     'email_text', 'email_version', 'hour', 'user_past_purchases', 'opened', 'clicked',
#     'user_country_ES', 'user_country_FR', 'user_country_UK', 'user_country_US',
#     'weekday_Friday', 'weekday_Monday', 'weekday_Saturday', 'weekday_Sunday',
#     'weekday_Thursday', 'weekday_Tuesday', 'weekday_Wednesday'
# ]

# # Step 3: Compute the correlation matrix
# corr_matrix = email_table[corr_columns].corr()

# # Step 4: Visualize the correlation matrix
# plt.figure(figsize=(14, 12))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, fmt='.2f')
# plt.title('Correlation Matrix of Email Campaign Features')
# plt.tight_layout()

# # Save the plot
# plt.savefig('correlation_matrix_sample.png')
# plt.close()

In [117]:
email_table['hour_bin'] = pd.cut(email_table['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])

In [118]:
email_table

,email_id,email_text,email_version,hour,user_past_purchases,opened,clicked,user_country_ES,user_country_FR,user_country_UK,user_country_US,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,hour_bin
0,85120,1,1,2,5,0,0,0,0,0,1,0,0,0,1,0,0,0,Night
1,966622,0,1,12,2,1,1,0,0,1,0,0,0,0,1,0,0,0,Morning
2,777221,0,1,11,2,0,0,0,0,0,1,0,0,0,0,0,0,1,Morning
3,493711,1,0,6,1,0,0,0,0,1,0,0,1,0,0,0,0,0,Night
4,106887,0,0,14,6,0,0,0,0,0,1,0,1,0,0,0,0,0,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,803504,1,1,4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,Night
99996,899722,0,1,7,1,0,0,0,0,0,1,0,0,1,0,0,0,0,Morning
99997,449610,0,1,11,6,0,0,0,0,1,0,0,0,1,0,0,0,0,Morning
99998,72497,1,0,10,0,0,0,0,0,1,0,0,1,0,0,0,0,0,Morning


In [119]:
onehot_encoder_hour_bin = OneHotEncoder()
hour_encoder = onehot_encoder_hour_bin.fit_transform(email_table[['hour_bin']])
hour_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100000 stored elements and shape (100000, 4)>

In [120]:
onehot_encoder_hour_bin.get_feature_names_out(['hour_bin'])

array(['hour_bin_Afternoon', 'hour_bin_Evening', 'hour_bin_Morning',
       'hour_bin_Night'], dtype=object)

In [121]:
hour_bin_encoded_df = pd.DataFrame(hour_encoder.toarray(),columns=onehot_encoder_hour_bin.get_feature_names_out(['hour_bin'])).astype(int)
hour_bin_encoded_df.head()

,hour_bin_Afternoon,hour_bin_Evening,hour_bin_Morning,hour_bin_Night
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0


In [122]:
email_table = pd.concat([email_table.drop(['hour','hour_bin'],axis=1),hour_bin_encoded_df],axis=1)

In [123]:
email_table

,email_id,email_text,email_version,user_past_purchases,opened,clicked,user_country_ES,user_country_FR,user_country_UK,user_country_US,...,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,hour_bin_Afternoon,hour_bin_Evening,hour_bin_Morning,hour_bin_Night
0,85120,1,1,5,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
1,966622,0,1,2,1,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
2,777221,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
3,493711,1,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
4,106887,0,0,6,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,803504,1,1,5,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
99996,899722,0,1,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
99997,449610,0,1,6,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
99998,72497,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [124]:
X = email_table.drop(['email_id', 'opened', 'clicked'], axis=1)
y = email_table['clicked']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
model = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
print(f"\nModel Accuracy: {accuracy:.2f}")
print(f"Model AUC: {auc:.2f}")


Model Accuracy: 0.98
Model AUC: 0.67
